In [1]:
import numpy as np
import pandas as pd

In [2]:
ames = pd.read_csv('./datasets/train.csv', na_values = '', keep_default_na= False)
ames_test = pd.read_csv('./datasets/test.csv', na_values = '', keep_default_na=False)
ames_test.shape

(878, 80)

In [3]:
def clean_data(df):
    data = df.copy()
    
    #Where Lot Frontage is null, set it to zero
    
    data['Lot Frontage'] = data['Lot Frontage'].where(data['Lot Frontage'].notna(), 0)
    
    #Where Masonry Area is Null, set to zero, where Type is null, set to 'None'
    
    data['Mas Vnr Area'] = data['Mas Vnr Area'].where(data['Mas Vnr Area'].notna(), 0)
    data['Mas Vnr Type'] = data['Mas Vnr Type'].where(data['Mas Vnr Type'].notna(), 'None')
    
    #Where the home has a basement, but exposure is null, set it to "No" for no exposure
    data['Bsmt Exposure'] = data['Bsmt Exposure'].where((data['Total Bsmt SF'] == 0) | (data['Bsmt Exposure'].notna()), 'No')
    
    #Basement Bathrooms, null = 0
    data['Bsmt Full Bath'] = data['Bsmt Full Bath'].where(data['Bsmt Full Bath'].notna(), 0)
    data['Bsmt Half Bath'] = data['Bsmt Half Bath'].where(data['Bsmt Half Bath'].notna(), 0)
    
    #Basement Finish Type 2 null -> NA
    data['BsmtFin Type 2'] = data['BsmtFin Type 2'].where(data['BsmtFin Type 2'].notna(), 'NA')
    
    #If the home has no garage, Garage Yr Blt is null. This won't work. Setting it to 0 will work.
    #Garage Yer Blt has too much collinearity with Year Built, so it won't be used in any model
    #It could still be used for feature engineering, however.
    data['Garage Yr Blt'] = data['Garage Yr Blt'].where(data['Garage Yr Blt'].notna(), 0)
    
    data['Garage Finish'] = data['Garage Finish'].where(data['Garage Finish'].notna(), 'NA')
    
    #In the testing data:
    
    data['Electrical'] = data['Electrical'].where(data['Electrical'].notna(), 'SBrkr') #Assume most common
    
    #The majority of nulls in the training and testing data are accounted for now. The rest of the data with null values will be dropped - TRAINING ONLY!
    
    data.dropna(inplace = True)
    
    #Drop rows where the "Age" feature becomes negative
    
    data.drop(data.loc[data['Yr Sold'] < data['Year Built']].index, inplace=True)
    
    #Cleaning up outliers that are either some extraordinary circumstance or data entry errors
    
    if('SalePrice' in data.columns):
        data.drop(data.loc[(data['SalePrice'] < 200_000) & (data['Gr Liv Area'] > 5000)].index, inplace = True)
    
    return data

In [4]:
ames = clean_data(ames)
ames.isna().sum()[ames.isna().sum() > 0]

Series([], dtype: int64)

In [5]:
#ames_test = clean_data(ames_test)
ames_test.isna().sum()[ames_test.isna().sum() > 0]

Lot Frontage     160
Mas Vnr Type       1
Mas Vnr Area       1
Electrical         1
Garage Yr Blt     45
Garage Finish      1
dtype: int64

In [6]:
ames_test[ames_test['Garage Finish'].isna()]['Garage Yr Blt']

764   NaN
Name: Garage Yr Blt, dtype: float64

In [7]:
ames_test['Electrical'].value_counts()

SBrkr    813
FuseA     48
FuseF     15
FuseP      1
Name: Electrical, dtype: int64

In [8]:
ames.shape

(2047, 81)

In [9]:
ames_test.shape

(878, 80)

In [10]:
ames.to_csv('./datasets/train_clean.csv')
ames_test.to_csv('./datasets/test_clean.csv')

In [11]:
ames.loc[(ames['SalePrice'] < 200_000) & (ames['Gr Liv Area'] > 5000)]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
